# GradientCast Quickstart

This notebook demonstrates the core functionality of the GradientCast SDK:

- Time series forecasting with `GradientCastFM`
- Deviation-based anomaly detection with `GradientCastPulseAD`
- Pattern-based anomaly detection with `GradientCastDenseAD`

## Installation

```bash
pip install gradientcast
```

In [ ]:
# Replace with your API key
GRADIENTCAST_API_KEY = "your-api-key-here"

---
## 1. Time Series Forecasting

Generate forecasts using `GradientCastFM`.

In [ ]:
from gradientcast import GradientCastFM

# Initialize client
fm = GradientCastFM(api_key=GRADIENTCAST_API_KEY)

# Your historical data
sales_data = [120, 135, 148, 142, 156, 163, 171, 168, 182, 195, 188, 201]

# Forecast 7 days ahead
result = fm.forecast(sales_data, horizon_len=7, freq="D")

print("Forecast:", [round(v, 1) for v in result["default"]])

In [ ]:
# Visualize the forecast
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(range(len(sales_data)), sales_data, 'b-o', label='Historical')
plt.plot(range(len(sales_data), len(sales_data) + 7), result["default"], 'r--o', label='Forecast')
plt.xlabel('Day')
plt.ylabel('Sales')
plt.title('Sales Forecast')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

---
## 2. Anomaly Detection (PulseAD)

`GradientCastPulseAD` detects significant deviations from expected behavior using dual-threshold logic.

**Note:** Historical data provides context, but anomaly detection is performed only on the latest data point(s).

In [ ]:
from gradientcast import GradientCastPulseAD

# Initialize client
ad = GradientCastPulseAD(api_key=GRADIENTCAST_API_KEY)

# Time series: earlier points provide context, last point is evaluated for anomaly
data = [
    {"timestamp": "12/01/2024, 12:00 PM", "value": 1500000},  # Context
    {"timestamp": "12/01/2024, 01:00 PM", "value": 1520000},  # Context
    {"timestamp": "12/01/2024, 02:00 PM", "value": 1510000},  # Context
    {"timestamp": "12/01/2024, 03:00 PM", "value": 1530000},  # Context
    {"timestamp": "12/01/2024, 04:00 PM", "value": 1525000},  # Context
    {"timestamp": "12/01/2024, 05:00 PM", "value": 850000},   # <-- Evaluated for anomaly
]

# Detect anomalies - uses history for context, evaluates latest point(s)
result = ad.detect({"user_count": data})

if result.has_anomaly:
    for a in result.anomalies:
        print(f"Anomaly detected at {a.timestamp}")
        print(f"  Actual: {a.actual_value:,} | Expected: {a.predicted_value:,.0f}")
        print(f"  Deviation: {a.percent_delta}")
else:
    print("No anomalies detected")

---
## 3. Anomaly Detection (DenseAD)

`GradientCastDenseAD` uses advanced density-based pattern analysis to detect anomalies with severity classification.

**Note:** Historical data provides context for pattern learning, but anomaly detection is performed only on the latest data point(s).

In [ ]:
from gradientcast import GradientCastDenseAD
from utils.synthetic_data import generate_ad_payload_data

# Initialize client
dense_ad = GradientCastDenseAD(api_key=GRADIENTCAST_API_KEY)

# Generate sample data with anomalies
# Earlier points serve as historical context, latest point(s) are evaluated
data = generate_ad_payload_data(n_points=100, inject_anomalies=True)

# Detect anomalies - uses history for context, evaluates latest point(s)
result = dense_ad.detect(data)

print(f"Alert Status: {result.alert_status}")
print(f"Severity: {result.alert_severity}")
print(f"Anomalies Found: {len(result.anomalies)}")

# Show details for confirmed anomalies
for point in result.anomalies[:3]:  # Show first 3
    print(f"  {point.timestamp}: value={point.value:,}, severity={point.magnitude.severity}")

In [ ]:
# Visualize the timeline with anomalies
import matplotlib.pyplot as plt

values = [p.value for p in result.timeline]
anomaly_mask = [p.confirmed_anomaly for p in result.timeline]
severity_colors = {'low': 'yellow', 'medium': 'orange', 'high': 'red', 'critical': 'darkred'}

plt.figure(figsize=(12, 4))
plt.plot(values, 'b-', alpha=0.7, label='Values')

# Mark anomalies
for i, (v, is_anom) in enumerate(zip(values, anomaly_mask)):
    if is_anom:
        sev = result.timeline[i].magnitude.severity
        plt.scatter(i, v, c=severity_colors.get(sev, 'red'), s=100, zorder=5, edgecolors='black')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Anomaly Detection Results')
plt.grid(alpha=0.3)
plt.show()

---
## Next Steps

- **[Forecasting Basics](forecasting/01_basics.ipynb)** - Deep dive into forecasting features
- **[Using Covariates](forecasting/02_covariates.ipynb)** - Improve accuracy with external variables
- **[PulseAD Guide](anomaly_detection/pulseAD/01_quickstart.ipynb)** - Threshold-based anomaly detection
- **[DensityAD Guide](anomaly_detection/densityAD/01_quickstart.ipynb)** - Pattern-based anomaly detection